# 1.5  Rflysim 多无人机控制

Rflysim 的多无人机功能支持同时仿真和控制多架无人机，适用于集群任务、协同导航等复杂场景。通过sdk可定义无人机的初始位置、类型及通信参数，并利用 Python API 或 ROS 实现分布式控制。例如，用户可通过循环创建多个 `PX4MavCtrler` 实例，分别连接不同端口，实现多机同步起飞、路径规划及传感器数据共享。  

该功能还支持动态环境交互，如天气模拟、光照变化及自定义地形，可验证无人机在 GPS 拒止环境下的自主导航能力。高级应用中，Rflysim 结合 PX4 和 ROS 框架，可构建分布式仿真平台，支持多机协同避障、任务分配等算法测试，为物流配送、搜索救援等场景提供高效验证工具。

1. **协同任务**  
   通过 ROS 或自定义协议实现多机通信，例如任务分配或避障协作。  
2. **分布式仿真**  
   在多台机器上运行多个 Rflysim 实例，模拟大规模集群场景。  
3. **可视化调试**  
   使用 RViz 可视化无人机状态和传感器数据。

Rflysim主要通过python代码动态添加多个无人机：


1. **连接多个无人机**  
   使用不同端口即可
   ```python
    import PX4MavCtrlV4 as PX4MavCtrl
    
    VehilceNum = 3
    START_INDEX=1
    MavList=[]
    # Create MAV instance
    for i in range(VehilceNum):
        ii=i+START_INDEX-1
        MavList=MavList+[PX4MavCtrl.PX4MavCtrler(1+i)]

   ```

通过以上步骤，您可以在 Rflysim 中快速搭建 3 无人机仿真环境，并通过 Python 实现灵活控制。

2. **质点多旋翼模型**

从模型精度的角度，使用高精度 6DOF 模型（CopterSim）+真实飞控系统（PX4）的软
/硬件在环仿真闭环的方式，能够有效提高模型可信度，从而减小仿真与真机实验的差距。
但是上述构架的代码运算量较为复杂，导致一台电脑上运行的无人机数量受到限制(SITL 软件在环<20 架)

为了提高单台电脑仿真集群飞机的数量，就需要降低模型精度并使用简化飞控模型。
因此本平台在 Python 下开发出了质点多旋翼模型，只需 Python 和 RflySim3D 两个软件
即可在单台电脑上实现百驾级别的无人机集群仿真。


本节就使用质点无人机模型，只是初始化稍有不同，其他都是一样的。

另外启动的时候，只需要启动UE环境即可，也就是启动Rflysim3D即可(PX4PSP\RflySim3D\RflySim3D.exe)

## 基本多机控制

In [1]:
import time
import math
import numpy as np
import PX4MavCtrlV4 as PX4MavCtrl
import UE4CtrlAPI
ue = UE4CtrlAPI.UE4CtrlAPI()

No Redis labs


In [2]:
#初始化8个无人机
VehilceNum = 8
START_INDEX=1
MavList=[]
# Create MAV instance
for i in range(VehilceNum):
    ii=i+START_INDEX-1
    MavList=MavList+[PX4MavCtrl.PX4MavCtrler(1+i)]

# sendUE4Cmd: RflySim3D API to modify scene display style
# Format: ue.sendUE4Cmd(cmd,windowID=-1), where cmd is a command string, windowID is the received window number (assuming multiple RflySim3D windows are opened at the same time), windowID =-1 means sent to all windows
# Augument: RflyChangeMapbyName command means to switch the map (scene), the following string is the map name, here will switch all open windows to the grass map
ue.sendUE4Cmd('RflyChangeMapbyName Grasslands')

In [3]:
#初始化无人机，并设置起飞点
InitPosList=[
    [-8.086,0,0,0],
    [-7.835,0,2,0],
    [-7.497,0,4,0],
    [-7.152,0,6,0],
    [-7.788,2,2,0],
    [-8.29,2,0,0],
    [-7.568,2,4,0],
    [-7.356,2,6,0],
]

time.sleep(2)
# Start MAV loop with UDP mode: Udp_Simple
# 就这一点和高精度无人机模型不同，其他都一样
for i in range(VehilceNum):
    MavList[i].initPointMassModel(InitPosList[i][0],InitPosList[i][1:4])

In [4]:
#起飞
# Get the takeoff position of each vehicle to the UE4 Map
# this can be adopted to obtain the global position of a vehicle in swarm simulation
time.sleep(2)
Error2UE4Map=[]
for i in range(VehilceNum):
    mav=MavList[i]
    Error2UE4Map = Error2UE4Map+[-np.array([mav.uavGlobalPos[0]-mav.uavPosNED[0],mav.uavGlobalPos[1]-mav.uavPosNED[1],mav.uavGlobalPos[2]-mav.uavPosNED[2]])]

# Enable vehicle number and trajectory display
ue.sendUE4Cmd('RflyChangeViewKeyCmd S')    
time.sleep(0.5)
ue.sendUE4Cmd('RflyChangeViewKeyCmd T')    

# fly to 10m high above its takeoff position
for i in range(VehilceNum):
    MavList[i].SendPosNED(0, 0, -10, 0) 
time.sleep(10)

In [5]:
#多无人机控制飞行
lastTime = time.time()
startTime = time.time()
timeInterval = 1/30.0 #here is 0.0333s (30Hz)
while True:
    lastTime = lastTime + timeInterval
    sleepTime = lastTime - time.time()
    if sleepTime > 0:
        time.sleep(sleepTime) # sleep until the desired clock
    else:
        lastTime = time.time()
    # The following code will be executed 30Hz (0.0333s)
    
    t=time.time()-startTime
    for j in range(VehilceNum):
        mav=MavList[j]
        
        # target position in UE4 map global frame
        if t<10:
            targetPosE=np.array([-0,0,-15])
        else:
            targetPosE=np.array([10*math.sin(t/2+math.pi/2)-10,10*math.sin(t/2.0),-15])
        
        # target position in vehilce takeoff frame
        targetPosE=targetPosE+Error2UE4Map[j]
        mav.SendPosNED(targetPosE[0],targetPosE[1],targetPosE[2],0)
    
    if t>200:
        break

In [6]:
#控制结束
for i in range(VehilceNum):
    MavList[i].EndPointMassModel() 

## 多无人机坐标系

每个无人机的坐标系都是自己的机体坐标系（NED），统一任务时，一般以1个无人机的坐标系为基准，进行坐标转换即可。
<img src="img/s1-5-1.png" width='600px' />



NED（North-East-Down，北东地）坐标系是无人机导航与控制中的核心参考系，其特点与应用如下：

### **1. NED定义与构成**
NED坐标系是一种**局部笛卡尔坐标系**，原点通常设在无人机的起飞点或任务起始位置：

- **X轴（N轴）**：指向地理正北方向；

- **Y轴（E轴）**：指向地理正东方向；

- **Z轴（D轴）**：垂直向下指向地心，遵循右手定则。

与机体坐标系（Body Frame）不同，NED坐标系是**固定在地面或惯性空间的导航坐标系**，用于全局位置和运动状态的描述。而机体坐标系则是固连在无人机上的坐标系，其X轴指向机头，Y轴指向右侧，Z轴垂直向下或向上（根据具体定义）。

---

### **2. 核心功能与应用**
#### **（1）导航与控制**
• **位置与速度基准**：NED坐标系将GPS提供的WGS-84经纬度转换为局部平面坐标，简化路径规划和高度控制。例如，无人机的位置偏移量（如北偏10米、东偏5米）可直接用NED坐标表示。

• **传感器数据融合**：IMU测量的加速度、陀螺仪角速度等机体坐标系数据，需通过坐标转换矩阵映射到NED坐标系，才能用于导航解算。

#### **（2）飞控系统兼容性**
在PX4等飞控系统中，NED坐标系是默认的导航参考系。无人机的初始位置（0,0,0）对应起飞点，所有控制指令（如目标位置、速度）均基于此坐标系生成。

---

### **3. 与其他坐标系的关系**
• **GPS（WGS-84）**：GPS的经纬度需通过地球模型转换到NED坐标系，以适配局部导航需求；

• **ECEF（地心地固系）**：NED是ECEF的局部简化版本，适用于小范围高精度导航，避免地心坐标系的计算复杂度。


参考文档
1. https://rflysim.com/doc/zh/10/RflySimSwarm.html